<a href="https://colab.research.google.com/github/Aniket99coder/EplaneEnergyOpt/blob/main/EnergyOptEplane-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install -q pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 33.7 MB/s eta 0:00:00


In [3]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_Time_Based", LpMinimize)

# Parameters (based on the image data)
battery_capacity = 21  # in kWh
soc_max_taxi = 3 / 100  # Convert percentage to fraction
soc_max_climb = 9 / 100
soc_max_descend = 3 / 100
soc_max_cruise = 48 / 100
roc_max_climb = 71  # in ft/min
altitude_cruise = 800  # in ft

# Time step in hours
delta_t = 1 / 60  # 1 minute in hours

# Total duration (in minutes) for each stage, assuming typical values within provided bounds
duration_taxi = 10  # Taxi for 10 minutes
duration_climb = 5  # Climb for 5 minutes
duration_descend = 5  # Descend for 5 minutes
duration_cruise = 30  # Cruise for 30 minutes

# Number of time intervals for each stage
n_taxi = int(duration_taxi / (delta_t * 60))
n_climb = int(duration_climb / (delta_t * 60))
n_descend = int(duration_descend / (delta_t * 60))
n_cruise = int(duration_cruise / (delta_t * 60))

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 0, 67
power_min_climb, power_max_climb = 0, 71
power_min_descend, power_max_descend = 0, 60
power_min_cruise, power_max_cruise = 0, 50

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(n_taxi)]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(n_climb)]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(n_descend)]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(n_cruise)]

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_descend]) + \
         lpSum([P * delta_t for P in P_cruise])

# Constraints

# Battery capacity constraint (total energy used must not exceed battery capacity)
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_descend]) + \
         lpSum([P * delta_t for P in P_cruise]) <= battery_capacity

# SoC drop constraints for each stage
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity <= soc_max_taxi
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity <= soc_max_climb
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity <= soc_max_descend
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity <= soc_max_cruise

# Additional constraints for rate of climb (for climb stage)
# Assuming a relationship or limitation based on max RoC in ft/min; placeholder constraint
model += lpSum(P_climb) / n_climb <= roc_max_climb  # This is a placeholder; adjust based on actual data

# Altitude constraint for cruise
# Assuming a relationship where cruise power should sustain altitude; placeholder
model += lpSum(P_cruise) / n_cruise >= 0.5 * altitude_cruise  # Placeholder relationship; replace as needed

# Solve the problem
model.solve()

# Print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("\nTotal Energy Consumption:", value(model.objective), "kWh")


Status: -1
Optimal Power Settings per Time Step:
Taxi Power: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] kW
Climb Power: [0.0, 0.0, 0.0, 0.0, 0.0] kW
Descend Power: [0.0, 0.0, 0.0, 0.0, 0.0] kW
Cruise Power: [11400.0, 50.0, 0.0, 0.0, 0.0, 50.0, 0.0, 0.0, 50.0, 50.0, 0.0, 0.0, 0.0, 0.0, 0.0, 50.0, 50.0, 50.0, 0.0, 50.0, 0.0, 0.0, 50.0, 50.0, 0.0, 50.0, 0.0, 50.0, 0.0, 0.0] kW

Total Energy Consumption: 200.0000000000001 kWh


## Simple Model (Few constraints )
- Objective: Minimize total energy consumption.
- Battery Capacity Constraint: Ensure the total energy does not exceed the battery capacity.
- Power Bounds for Each Stage: Ensure the power for each stage is within specified minimum and maximum bounds.


In [4]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_SoC_Constraints", LpMinimize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage

# Time step in hours (1 minute in hours)
delta_t = 1 / 60

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 1, 10  # Set minimum power to ensure operation
power_min_climb, power_max_climb = 50, 70  # Set minimum power for climb phase
power_min_cruise, power_max_cruise = 10, 50  # Minimum cruise power to sustain flight
power_min_descend, power_max_descend = 5, 25  # Descent might require less power

# Duration bounds (in hours) for each stage
duration_taxi_min, duration_taxi_max = 0.003, 0.277  # Min and max duration for taxi in hours
duration_climb_min, duration_climb_max = 0.006, 0.047
duration_cruise_min, duration_cruise_max = 0.003, 0.503
duration_descend_min, duration_descend_max = 0.005, 0.052

# SoC drop limits (percent of battery capacity) for each stage
soc_max_drop_taxi = 3  # Maximum 3% drop in Taxi
soc_max_drop_climb = 9  # Maximum 9% drop in Climb
soc_max_drop_cruise = 48  # Maximum 48% drop in Cruise
soc_max_drop_descend = 3  # Maximum 3% drop in Descend

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(int(duration_taxi_max / delta_t))]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(int(duration_climb_max / delta_t))]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(int(duration_cruise_max / delta_t))]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(int(duration_descend_max / delta_t))]

# Duration variables for each stage
d_taxi = LpVariable("Duration_Taxi", lowBound=duration_taxi_min, upBound=duration_taxi_max)
d_climb = LpVariable("Duration_Climb", lowBound=duration_climb_min, upBound=duration_climb_max)
d_cruise = LpVariable("Duration_Cruise", lowBound=duration_cruise_min, upBound=duration_cruise_max)
d_descend = LpVariable("Duration_Descend", lowBound=duration_descend_min, upBound=duration_descend_max)

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend])

# Constraints

# Battery capacity constraint (total energy used must not exceed battery capacity)
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend]) <= battery_capacity, "Battery_Capacity"

# Duration constraints for each stage
model += lpSum(delta_t for _ in P_taxi) == d_taxi, "Duration_Taxi"
model += lpSum(delta_t for _ in P_climb) == d_climb, "Duration_Climb"
model += lpSum(delta_t for _ in P_cruise) == d_cruise, "Duration_Cruise"
model += lpSum(delta_t for _ in P_descend) == d_descend, "Duration_Descend"

# Minimum operational power constraints to avoid trivial zero power usage
for t in range(int(duration_taxi_max / delta_t)):
    model += P_taxi[t] >= power_min_taxi, f"Min_Power_Taxi_{t}"

for t in range(int(duration_climb_max / delta_t)):
    model += P_climb[t] >= power_min_climb, f"Min_Power_Climb_{t}"

for t in range(int(duration_cruise_max / delta_t)):
    model += P_cruise[t] >= power_min_cruise, f"Min_Power_Cruise_{t}"

for t in range(int(duration_descend_max / delta_t)):
    model += P_descend[t] >= power_min_descend, f"Min_Power_Descend_{t}"

# SoC drop constraints applied to each stage
# Calculate cumulative energy used in each stage and ensure it doesn't exceed max SoC drop

# Taxi stage SoC drop constraint
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity * 100 <= soc_max_drop_taxi, "SoC_Drop_Taxi"
# Climb stage SoC drop constraint
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 <= soc_max_drop_climb, "SoC_Drop_Climb"
# Cruise stage SoC drop constraint
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity * 100 <= soc_max_drop_cruise, "SoC_Drop_Cruise"
# Descend stage SoC drop constraint
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity * 100 <= soc_max_drop_descend, "SoC_Drop_Descend"


# Solve the problem
model.solve()

# Calculate and print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")

# Print Duration in both hours and minutes
print("\nOptimal Duration Settings (in hours and minutes):")
print("Taxi Duration:", value(d_taxi), "hours (", round(value(d_taxi) * 60, 2), "minutes)")
print("Climb Duration:", value(d_climb), "hours (", round(value(d_climb) * 60, 2), "minutes)")
print("Cruise Duration:", value(d_cruise), "hours (", round(value(d_cruise) * 60, 2), "minutes)")
print("Descend Duration:", value(d_descend), "hours (", round(value(d_descend) * 60, 2), "minutes)")

# Calculate and display the SoC at each time step
soc = initial_soc

print("\nState of Charge (SoC) Levels per Time Step:")
print("Stage\tTime Step\tPower (kW)\tSoC (%)")

# Function to calculate SoC
def calculate_soc(power_values, duration):
    global soc
    for t, power in enumerate(power_values):
        power_value = value(power)
        soc -= (power_value * delta_t / battery_capacity) * 100
        print(f"{duration}\t{t+1}\t\t{power_value:.2f}\t\t{soc:.2f}")

# Display SoC levels for each stage
calculate_soc(P_taxi, "Taxi")
calculate_soc(P_climb, "Climb")
calculate_soc(P_cruise, "Cruise")
calculate_soc(P_descend, "Descend")

print("\nTotal Energy Consumption:", value(model.objective), "kWh")


Status: 1
Optimal Power Settings per Time Step:
Taxi Power: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] kW
Climb Power: [50.0, 50.0] kW
Cruise Power: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] kW
Descend Power: [5.0, 5.0, 5.0] kW

Optimal Duration Settings (in hours and minutes):
Taxi Duration: 0.26666667 hours ( 16.0 minutes)
Climb Duration: 0.033333333 hours ( 2.0 minutes)
Cruise Duration: 0.5 hours ( 30.0 minutes)
Descend Duration: 0.05 hours ( 3.0 minutes)

State of Charge (SoC) Levels per Time Step:
Stage	Time Step	Power (kW)	SoC (%)
Taxi	1		1.00		99.92
Taxi	2		1.00		99.84
Taxi	3		1.00		99.76
Taxi	4		1.00		99.68
Taxi	5		1.00		99.60
Taxi	6		1.00		99.52
Taxi	7		1.00		99.44
Taxi	8		1.00		99.37
Taxi	9		1.00		99.29
Taxi	10		1.00		99.21
Taxi	11		1.00		99.13
Taxi	12		1.00		99.05
Taxi	13		1.00		98.97
Taxi	14		1.00		9

In [5]:
# Print specific SoC drop constraints
for name, constraint in model.constraints.items():
    if "SoC_Drop" in name:
        print(f"{name}: {constraint}")


SoC_Drop_Taxi: 0.07936507936507936*Power_Taxi_0 + 0.07936507936507936*Power_Taxi_1 + 0.07936507936507936*Power_Taxi_10 + 0.07936507936507936*Power_Taxi_11 + 0.07936507936507936*Power_Taxi_12 + 0.07936507936507936*Power_Taxi_13 + 0.07936507936507936*Power_Taxi_14 + 0.07936507936507936*Power_Taxi_15 + 0.07936507936507936*Power_Taxi_2 + 0.07936507936507936*Power_Taxi_3 + 0.07936507936507936*Power_Taxi_4 + 0.07936507936507936*Power_Taxi_5 + 0.07936507936507936*Power_Taxi_6 + 0.07936507936507936*Power_Taxi_7 + 0.07936507936507936*Power_Taxi_8 + 0.07936507936507936*Power_Taxi_9 <= 3.0
SoC_Drop_Climb: 0.07936507936507936*Power_Climb_0 + 0.07936507936507936*Power_Climb_1 <= 9.0
SoC_Drop_Cruise: 0.07936507936507936*Power_Cruise_0 + 0.07936507936507936*Power_Cruise_1 + 0.07936507936507936*Power_Cruise_10 + 0.07936507936507936*Power_Cruise_11 + 0.07936507936507936*Power_Cruise_12 + 0.07936507936507936*Power_Cruise_13 + 0.07936507936507936*Power_Cruise_14 + 0.07936507936507936*Power_Cruise_15 + 0.

In [6]:
# Print the values of all decision variables
print("\nValues of Decision Variables:")
for variable in model.variables():
    print(f"{variable.name} = {value(variable)}")


Values of Decision Variables:
Duration_Climb = 0.033333333
Duration_Cruise = 0.5
Duration_Descend = 0.05
Duration_Taxi = 0.26666667
Power_Climb_0 = 50.0
Power_Climb_1 = 50.0
Power_Cruise_0 = 10.0
Power_Cruise_1 = 10.0
Power_Cruise_10 = 10.0
Power_Cruise_11 = 10.0
Power_Cruise_12 = 10.0
Power_Cruise_13 = 10.0
Power_Cruise_14 = 10.0
Power_Cruise_15 = 10.0
Power_Cruise_16 = 10.0
Power_Cruise_17 = 10.0
Power_Cruise_18 = 10.0
Power_Cruise_19 = 10.0
Power_Cruise_2 = 10.0
Power_Cruise_20 = 10.0
Power_Cruise_21 = 10.0
Power_Cruise_22 = 10.0
Power_Cruise_23 = 10.0
Power_Cruise_24 = 10.0
Power_Cruise_25 = 10.0
Power_Cruise_26 = 10.0
Power_Cruise_27 = 10.0
Power_Cruise_28 = 10.0
Power_Cruise_29 = 10.0
Power_Cruise_3 = 10.0
Power_Cruise_4 = 10.0
Power_Cruise_5 = 10.0
Power_Cruise_6 = 10.0
Power_Cruise_7 = 10.0
Power_Cruise_8 = 10.0
Power_Cruise_9 = 10.0
Power_Descend_0 = 5.0
Power_Descend_1 = 5.0
Power_Descend_2 = 5.0
Power_Taxi_0 = 1.0
Power_Taxi_1 = 1.0
Power_Taxi_10 = 1.0
Power_Taxi_11 = 1.0
P

In [7]:
# Solve the model
model.solve()

# Check constraint satisfaction
print("\nConstraint Satisfaction Check:")
for name, constraint in model.constraints.items():
    lhs_value = constraint.value()
    if (constraint.sense == -1 and lhs_value > constraint.constant) or \
       (constraint.sense == 1 and lhs_value < constraint.constant) or \
       (constraint.sense == 0 and lhs_value != constraint.constant):
        print(f"Constraint '{name}' is not satisfied: LHS = {lhs_value}, RHS = {constraint.constant}")
    else:
        print(f"Constraint '{name}' is satisfied: LHS = {lhs_value}, RHS = {constraint.constant}")



Constraint Satisfaction Check:
Constraint 'Battery_Capacity' is not satisfied: LHS = -13.816666666666668, RHS = -21.0
Constraint 'Duration_Taxi' is not satisfied: LHS = -3.333333331578814e-09, RHS = 0.26666666666666666
Constraint 'Duration_Climb' is not satisfied: LHS = 3.333333331578814e-10, RHS = 0.03333333333333333
Constraint 'Duration_Cruise' is not satisfied: LHS = -5.551115123125783e-17, RHS = 0.49999999999999994
Constraint 'Duration_Descend' is not satisfied: LHS = 0.0, RHS = 0.05
Constraint 'Min_Power_Taxi_0' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_1' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_2' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_3' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_4' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_5' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_6' is satisfied: LHS = 0.0, RHS = -1
Constraint 'Min_Power_Taxi_7' is satisfied: LHS = 0.0, RHS

## Model 2 ( Reach X altitude before cruising)

In [8]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_SoC_Constraints", LpMinimize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_drop_rate = 2  # Minimum SoC drop rate in % per minute for climb

# Time step in hours (1 minute in hours)
delta_t = 1 / 60

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 1, 10
power_min_climb, power_max_climb = 2, 70
power_min_cruise, power_max_cruise = 10, 50
power_min_descend, power_max_descend = 5, 25

# Duration bounds (in hours) for each stage
duration_taxi_min, duration_taxi_max = 0.003, 0.277
duration_climb_min, duration_climb_max = 0.006, 0.047
duration_cruise_min, duration_cruise_max = 0.003, 0.503
duration_descend_min, duration_descend_max = 0.005, 0.052

# SoC drop limits for each stage
soc_max_drop_taxi = 3
soc_max_drop_climb = 9
soc_max_drop_cruise = 48
soc_max_drop_descend = 3

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(int(duration_taxi_max / delta_t))]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(int(duration_climb_max / delta_t))]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(int(duration_cruise_max / delta_t))]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(int(duration_descend_max / delta_t))]

# Duration variables for each stage
d_taxi = LpVariable("Duration_Taxi", lowBound=duration_taxi_min, upBound=duration_taxi_max)
d_climb = LpVariable("Duration_Climb", lowBound=duration_climb_min, upBound=duration_climb_max)
d_cruise = LpVariable("Duration_Cruise", lowBound=duration_cruise_min, upBound=duration_cruise_max)
d_descend = LpVariable("Duration_Descend", lowBound=duration_descend_min, upBound=duration_descend_max)

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend])

# Constraints

# Battery capacity constraint
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend]) <= battery_capacity, "Battery_Capacity"

# Duration constraints for each stage
model += lpSum(delta_t for _ in P_taxi) == d_taxi, "Duration_Taxi"
model += lpSum(delta_t for _ in P_climb) == d_climb, "Duration_Climb"
model += lpSum(delta_t for _ in P_cruise) == d_cruise, "Duration_Cruise"
model += lpSum(delta_t for _ in P_descend) == d_descend, "Duration_Descend"

# Minimum operational power constraints
for t in range(int(duration_taxi_max / delta_t)):
    model += P_taxi[t] >= power_min_taxi, f"Min_Power_Taxi_{t}"

for t in range(int(duration_climb_max / delta_t)):
    model += P_climb[t] >= power_min_climb, f"Min_Power_Climb_{t}"

for t in range(int(duration_cruise_max / delta_t)):
    model += P_cruise[t] >= power_min_cruise, f"Min_Power_Cruise_{t}"

for t in range(int(duration_descend_max / delta_t)):
    model += P_descend[t] >= power_min_descend, f"Min_Power_Descend_{t}"

# SoC drop constraints applied to each stage
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity * 100 <= soc_max_drop_taxi, "SoC_Drop_Taxi"
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 <= soc_max_drop_climb, "SoC_Drop_Climb"
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity * 100 <= soc_max_drop_cruise, "SoC_Drop_Cruise"
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity * 100 <= soc_max_drop_descend, "SoC_Drop_Descend"

# Altitude constraint: Ensure SoC drop during climb is enough to reach target altitude
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 * altitude_rate >= altitude_target, "Altitude_Climb_Target"

# Minimum SoC drop rate constraint for climb to achieve rate of altitude gain
for t in range(int(duration_climb_max / delta_t)):
    model += (P_climb[t] * delta_t / battery_capacity) * 100 >= min_soc_drop_rate, f"Min_SoC_Drop_Rate_Climb_{t}"

# Solve the problem
model.solve()

# Print results
# Calculate and print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")

# Print Duration in both hours and minutes
print("\nOptimal Duration Settings (in hours and minutes):")
print("Taxi Duration:", value(d_taxi), "hours (", round(value(d_taxi) * 60, 2), "minutes)")
print("Climb Duration:", value(d_climb), "hours (", round(value(d_climb) * 60, 2), "minutes)")
print("Cruise Duration:", value(d_cruise), "hours (", round(value(d_cruise) * 60, 2), "minutes)")
print("Descend Duration:", value(d_descend), "hours (", round(value(d_descend) * 60, 2), "minutes)")

# Calculate and display the SoC at each time step
soc = initial_soc

print("\nState of Charge (SoC) Levels per Time Step:")
print("Stage\tTime Step\tPower (kW)\tSoC (%)")

# Function to calculate SoC
def calculate_soc(power_values, duration):
    global soc
    for t, power in enumerate(power_values):
        power_value = value(power)
        soc -= (power_value * delta_t / battery_capacity) * 100
        print(f"{duration}\t{t+1}\t\t{power_value:.2f}\t\t{soc:.2f}")

# Display SoC levels for each stage
calculate_soc(P_taxi, "Taxi")
calculate_soc(P_climb, "Climb")
calculate_soc(P_cruise, "Cruise")
calculate_soc(P_descend, "Descend")

print("\nTotal Energy Consumption:", value(model.objective), "kWh")

Status: 1
Optimal Power Settings per Time Step:
Taxi Power: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] kW
Climb Power: [70.0, 39.2] kW
Cruise Power: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] kW
Descend Power: [5.0, 5.0, 5.0] kW

Optimal Duration Settings (in hours and minutes):
Taxi Duration: 0.26666667 hours ( 16.0 minutes)
Climb Duration: 0.033333333 hours ( 2.0 minutes)
Cruise Duration: 0.5 hours ( 30.0 minutes)
Descend Duration: 0.05 hours ( 3.0 minutes)

State of Charge (SoC) Levels per Time Step:
Stage	Time Step	Power (kW)	SoC (%)
Taxi	1		1.00		99.92
Taxi	2		1.00		99.84
Taxi	3		1.00		99.76
Taxi	4		1.00		99.68
Taxi	5		1.00		99.60
Taxi	6		1.00		99.52
Taxi	7		1.00		99.44
Taxi	8		1.00		99.37
Taxi	9		1.00		99.29
Taxi	10		1.00		99.21
Taxi	11		1.00		99.13
Taxi	12		1.00		99.05
Taxi	13		1.00		98.97
Taxi	14		1.00		9

# 2. Using the whiskers from real data for power for `Model 2`

Power Statistics by Phase:

Phase: Cruise
- Median Power: 19.595 kW
- Average Power: 19.860 kW
- Minimum Power: 11.535 kW
- Maximum Power: 36.600 kW
- Q1 (25th Percentile): 17.293 kW
- Q3 (75th Percentile): 21.673 kW

----------------------------------------

Phase: Taxi
- Median Power: 1.150 kW
- Average Power: 1.218 kW
- Minimum Power: 0.000 kW
- Maximum Power: 8.954 kW
- Q1 (25th Percentile): 0.645 kW
- Q3 (75th Percentile): 1.477 kW

----------------------------------------

Phase: CLimb
- Median Power: 45.813 kW
- Average Power: 45.339 kW
- Minimum Power: 30.753 kW
- Maximum Power: 57.227 kW
- Q1 (25th Percentile): 42.715 kW
- Q3 (75th Percentile): 47.810 kW

----------------------------------------

Phase: Landing
- Median Power: 1.036 kW
- Average Power: 1.966 kW
- Minimum Power: 0.000 kW
- Maximum Power: 24.310 kW
- Q1 (25th Percentile): 0.338 kW
- Q3 (75th Percentile): 2.656 kW

----------------------------------------

In [11]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_SoC_Constraints", LpMinimize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_drop_rate = 2  # Minimum SoC drop rate in % per minute for climb

# Time step in hours (1 minute in hours)
delta_t = 1 / 60

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 0.5, 2
power_min_climb, power_max_climb = 40, 50
power_min_cruise, power_max_cruise = 15, 22
power_min_descend, power_max_descend = 0, 3

# Duration bounds (in hours) for each stage
duration_taxi_min, duration_taxi_max = 0.003, 0.122
duration_climb_min, duration_climb_max = 0.006, 0.033
duration_cruise_min, duration_cruise_max = 0.003, 0.029
duration_descend_min, duration_descend_max = 0.005, 0.05

# SoC drop limits for each stage
soc_max_drop_taxi = 3
soc_max_drop_climb = 9
soc_max_drop_cruise = 48
soc_max_drop_descend = 3

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(int(duration_taxi_max / delta_t))]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(int(duration_climb_max / delta_t))]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(int(duration_cruise_max / delta_t))]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(int(duration_descend_max / delta_t))]

# Duration variables for each stage
d_taxi = LpVariable("Duration_Taxi", lowBound=duration_taxi_min, upBound=duration_taxi_max)
d_climb = LpVariable("Duration_Climb", lowBound=duration_climb_min, upBound=duration_climb_max)
d_cruise = LpVariable("Duration_Cruise", lowBound=duration_cruise_min, upBound=duration_cruise_max)
d_descend = LpVariable("Duration_Descend", lowBound=duration_descend_min, upBound=duration_descend_max)

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend])

# Constraints

# Battery capacity constraint
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend]) <= battery_capacity, "Battery_Capacity"

# Duration constraints for each stage
model += lpSum(delta_t for _ in P_taxi) == d_taxi, "Duration_Taxi"
model += lpSum(delta_t for _ in P_climb) == d_climb, "Duration_Climb"
model += lpSum(delta_t for _ in P_cruise) == d_cruise, "Duration_Cruise"
model += lpSum(delta_t for _ in P_descend) == d_descend, "Duration_Descend"

# Minimum operational power constraints
for t in range(int(duration_taxi_max / delta_t)):
    model += P_taxi[t] >= power_min_taxi, f"Min_Power_Taxi_{t}"

for t in range(int(duration_climb_max / delta_t)):
    model += P_climb[t] >= power_min_climb, f"Min_Power_Climb_{t}"

for t in range(int(duration_cruise_max / delta_t)):
    model += P_cruise[t] >= power_min_cruise, f"Min_Power_Cruise_{t}"

for t in range(int(duration_descend_max / delta_t)):
    model += P_descend[t] >= power_min_descend, f"Min_Power_Descend_{t}"

# SoC drop constraints applied to each stage
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity * 100 <= soc_max_drop_taxi, "SoC_Drop_Taxi"
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 <= soc_max_drop_climb, "SoC_Drop_Climb"
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity * 100 <= soc_max_drop_cruise, "SoC_Drop_Cruise"
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity * 100 <= soc_max_drop_descend, "SoC_Drop_Descend"

# Altitude constraint: Ensure SoC drop during climb is enough to reach target altitude
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 * altitude_rate >= altitude_target, "Altitude_Climb_Target"

# Minimum SoC drop rate constraint for climb to achieve rate of altitude gain
for t in range(int(duration_climb_max / delta_t)):
    model += (P_climb[t] * delta_t / battery_capacity) * 100 >= min_soc_drop_rate, f"Min_SoC_Drop_Rate_Climb_{t}"

# Solve the problem
model.solve()

# Print results
# Calculate and print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")

# Print Duration in both hours and minutes
print("\nOptimal Duration Settings (in hours and minutes):")
print("Taxi Duration:", value(d_taxi), "hours (", round(value(d_taxi) * 60, 2), "minutes)")
print("Climb Duration:", value(d_climb), "hours (", round(value(d_climb) * 60, 2), "minutes)")
print("Cruise Duration:", value(d_cruise), "hours (", round(value(d_cruise) * 60, 2), "minutes)")
print("Descend Duration:", value(d_descend), "hours (", round(value(d_descend) * 60, 2), "minutes)")

# Calculate and display the SoC at each time step
soc = initial_soc

print("\nState of Charge (SoC) Levels per Time Step:")
print("Stage\tTime Step\tPower (kW)\tSoC (%)")

# Function to calculate SoC
def calculate_soc(power_values, duration):
    global soc
    for t, power in enumerate(power_values):
        power_value = value(power)
        soc -= (power_value * delta_t / battery_capacity) * 100
        print(f"{duration}\t{t+1}\t\t{power_value:.2f}\t\t{soc:.2f}")

# Display SoC levels for each stage
calculate_soc(P_taxi, "Taxi")
calculate_soc(P_climb, "Climb")
calculate_soc(P_cruise, "Cruise")
calculate_soc(P_descend, "Descend")

print("\nTotal Energy Consumption:", value(model.objective), "kWh")

Status: -1
Optimal Power Settings per Time Step:
Taxi Power: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5] kW
Climb Power: [109.2] kW
Cruise Power: [15.0] kW
Descend Power: [0.0, 0.0, 0.0] kW

Optimal Duration Settings (in hours and minutes):
Taxi Duration: 0.11666667 hours ( 7.0 minutes)
Climb Duration: 0.006 hours ( 0.36 minutes)
Cruise Duration: 0.003 hours ( 0.18 minutes)
Descend Duration: 0.005 hours ( 0.3 minutes)

State of Charge (SoC) Levels per Time Step:
Stage	Time Step	Power (kW)	SoC (%)
Taxi	1		0.50		99.96
Taxi	2		0.50		99.92
Taxi	3		0.50		99.88
Taxi	4		0.50		99.84
Taxi	5		0.50		99.80
Taxi	6		0.50		99.76
Taxi	7		0.50		99.72
Climb	1		109.20		91.06
Cruise	1		15.00		89.87
Descend	1		0.00		89.87
Descend	2		0.00		89.87
Descend	3		0.00		89.87

Total Energy Consumption: 2.1283333333333334 kWh


In [74]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_SoC_Constraints", LpMinimize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1000  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_drop_rate = 2  # Minimum SoC drop rate in % per minute for climb

# Time step in hours (1 minute in hours)
delta_t = 1 / 60

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 1, 10
power_min_climb, power_max_climb = 2, 70
power_min_cruise, power_max_cruise = 10, 50
power_min_descend, power_max_descend = 5, 25

# Duration bounds (in hours) for each stage
duration_taxi_min, duration_taxi_max = 0.003, 0.277
duration_climb_min, duration_climb_max = 0.006, 0.047
duration_cruise_min, duration_cruise_max = 0.003, 0.503
duration_descend_min, duration_descend_max = 0.005, 0.052

# SoC drop limits for each stage
soc_max_drop_taxi = 3
soc_max_drop_climb = 9
soc_max_drop_cruise = 48
soc_max_drop_descend = 3

# Create decision variables for each time interval in each stage
P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(int(duration_taxi_max / delta_t))]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(int(duration_climb_max / delta_t))]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(int(duration_cruise_max / delta_t))]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(int(duration_descend_max / delta_t))]

# Duration variables for each stage
d_taxi = LpVariable("Duration_Taxi", lowBound=duration_taxi_min, upBound=duration_taxi_max)
d_climb = LpVariable("Duration_Climb", lowBound=duration_climb_min, upBound=duration_climb_max)
d_cruise = LpVariable("Duration_Cruise", lowBound=duration_cruise_min, upBound=duration_cruise_max)
d_descend = LpVariable("Duration_Descend", lowBound=duration_descend_min, upBound=duration_descend_max)

# Objective: Minimize total energy consumption across all time intervals and stages
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend])

# Constraints

# Battery capacity constraint
model += lpSum([P * delta_t for P in P_taxi]) + \
         lpSum([P * delta_t for P in P_climb]) + \
         lpSum([P * delta_t for P in P_cruise]) + \
         lpSum([P * delta_t for P in P_descend]) <= battery_capacity, "Battery_Capacity"

# Duration constraints for each stage
model += lpSum(delta_t for _ in P_taxi) == d_taxi, "Duration_Taxi"
model += lpSum(delta_t for _ in P_climb) == d_climb, "Duration_Climb"
model += lpSum(delta_t for _ in P_cruise) == d_cruise, "Duration_Cruise"
model += lpSum(delta_t for _ in P_descend) == d_descend, "Duration_Descend"

# Minimum operational power constraints
for t in range(int(duration_taxi_max / delta_t)):
    model += P_taxi[t] >= power_min_taxi, f"Min_Power_Taxi_{t}"

for t in range(int(duration_climb_max / delta_t)):
    model += P_climb[t] >= power_min_climb, f"Min_Power_Climb_{t}"

for t in range(int(duration_cruise_max / delta_t)):
    model += P_cruise[t] >= power_min_cruise, f"Min_Power_Cruise_{t}"

for t in range(int(duration_descend_max / delta_t)):
    model += P_descend[t] >= power_min_descend, f"Min_Power_Descend_{t}"

# SoC drop constraints applied to each stage
model += lpSum([P * delta_t for P in P_taxi]) / battery_capacity * 100 <= soc_max_drop_taxi, "SoC_Drop_Taxi"
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 <= soc_max_drop_climb, "SoC_Drop_Climb"
model += lpSum([P * delta_t for P in P_cruise]) / battery_capacity * 100 <= soc_max_drop_cruise, "SoC_Drop_Cruise"
model += lpSum([P * delta_t for P in P_descend]) / battery_capacity * 100 <= soc_max_drop_descend, "SoC_Drop_Descend"

# Altitude constraint: Ensure SoC drop during climb is enough to reach target altitude
model += lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100 * altitude_rate >= altitude_target, "Altitude_Climb_Target"

# Minimum SoC drop rate constraint for climb to achieve rate of altitude gain
for t in range(int(duration_climb_max / delta_t)):
    model += (P_climb[t] * delta_t / battery_capacity) * 100 >= min_soc_drop_rate, f"Min_SoC_Drop_Rate_Climb_{t}"

# Solve the problem
model.solve()

# Print results
# Calculate and print results
print("Status:", model.status)
print("Optimal Power Settings per Time Step:")
print("Taxi Power:", [value(P) for P in P_taxi], "kW")
print("Climb Power:", [value(P) for P in P_climb], "kW")
print("Cruise Power:", [value(P) for P in P_cruise], "kW")
print("Descend Power:", [value(P) for P in P_descend], "kW")

# Print Duration in both hours and minutes
print("\nOptimal Duration Settings (in hours and minutes):")
print("Taxi Duration:", value(d_taxi), "hours (", round(value(d_taxi) * 60, 2), "minutes)")
print("Climb Duration:", value(d_climb), "hours (", round(value(d_climb) * 60, 2), "minutes)")
print("Cruise Duration:", value(d_cruise), "hours (", round(value(d_cruise) * 60, 2), "minutes)")
print("Descend Duration:", value(d_descend), "hours (", round(value(d_descend) * 60, 2), "minutes)")

# Calculate and display the SoC at each time step
soc = initial_soc

print("\nState of Charge (SoC) Levels per Time Step:")
print("Stage\tTime Step\tPower (kW)\tSoC (%)")

# Function to calculate SoC
def calculate_soc(power_values, duration):
    global soc
    for t, power in enumerate(power_values):
        power_value = value(power)
        soc -= (power_value * delta_t / battery_capacity) * 100
        print(f"{duration}\t{t+1}\t\t{power_value:.2f}\t\t{soc:.2f}")

# Display SoC levels for each stage
calculate_soc(P_taxi, "Taxi")
calculate_soc(P_climb, "Climb")
calculate_soc(P_cruise, "Cruise")
calculate_soc(P_descend, "Descend")

print("\nTotal Energy Consumption:", value(model.objective), "kWh")

Status: 1
Optimal Power Settings per Time Step:
Taxi Power: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0] kW
Climb Power: [58.8, 25.2] kW
Cruise Power: [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] kW
Descend Power: [5.0, 5.0, 5.0] kW

Optimal Duration Settings (in hours and minutes):
Taxi Duration: 0.26666667 hours ( 16.0 minutes)
Climb Duration: 0.033333333 hours ( 2.0 minutes)
Cruise Duration: 0.5 hours ( 30.0 minutes)
Descend Duration: 0.05 hours ( 3.0 minutes)

State of Charge (SoC) Levels per Time Step:
Stage	Time Step	Power (kW)	SoC (%)
Taxi	1		1.00		99.92
Taxi	2		1.00		99.84
Taxi	3		1.00		99.76
Taxi	4		1.00		99.68
Taxi	5		1.00		99.60
Taxi	6		1.00		99.52
Taxi	7		1.00		99.44
Taxi	8		1.00		99.37
Taxi	9		1.00		99.29
Taxi	10		1.00		99.21
Taxi	11		1.00		99.13
Taxi	12		1.00		99.05
Taxi	13		1.00		98.97
Taxi	14		1.00		9

## Working model for circuit flights

In [1]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

# Define the optimization problem
model = LpProblem("Energy_Optimization_with_Circuits", LpMaximize)

# Parameters
battery_capacity = 21  # in kWh
initial_soc = 100  # Initial State of Charge in percentage
altitude_target = 1300  # Target altitude to reach before cruising in feet
altitude_rate = 150  # Feet gained per 1% SoC drop during climb
min_soc_for_circuit = 20  # Minimum SoC required to start a circuit in percentage
max_circuits = 20  # Maximum number of circuits allowed
min_soc_drop_rate = 2  # Minimum SoC drop rate in % per minute for climb

# Time step in hours (1 minute in hours)
delta_t = 1 / 60

# Power bounds (in kW) for each stage
power_min_taxi, power_max_taxi = 0.5, 2
power_min_climb, power_max_climb = 40, 50
power_min_cruise, power_max_cruise = 15, 22
power_min_descend, power_max_descend = 0, 3

# Decision variables
num_circuits = LpVariable("Number_of_Circuits", lowBound=0, upBound=max_circuits, cat="Integer")
soc_remaining = LpVariable.dicts("SoC_Remaining", range(max_circuits + 1), lowBound=0, upBound=100)
soc_drop_actual = LpVariable.dicts("SoC_Drop_Actual", range(1, max_circuits + 1), lowBound=0, upBound=100)
is_circuit_active = LpVariable.dicts("Is_Circuit_Active", range(1, max_circuits + 1), cat="Binary")

P_taxi = [LpVariable(f"Power_Taxi_{t}", lowBound=power_min_taxi, upBound=power_max_taxi) for t in range(int(0.122 / delta_t))]
P_climb = [LpVariable(f"Power_Climb_{t}", lowBound=power_min_climb, upBound=power_max_climb) for t in range(int(0.033 / delta_t))]
P_cruise = [LpVariable(f"Power_Cruise_{t}", lowBound=power_min_cruise, upBound=power_max_cruise) for t in range(int(0.029 / delta_t))]
P_descend = [LpVariable(f"Power_Descend_{t}", lowBound=power_min_descend, upBound=power_max_descend) for t in range(int(0.05 / delta_t))]

# Objective: Maximize the number of circuits
model += lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Objective"

# Initial SoC constraint
model += soc_remaining[0] == initial_soc, "Initial_SoC"

# Calculate SoC drop for a single circuit
soc_drop_taxi = lpSum([P * delta_t for P in P_taxi]) / battery_capacity * 100
soc_drop_climb = lpSum([P * delta_t for P in P_climb]) / battery_capacity * 100
soc_drop_cruise = lpSum([P * delta_t for P in P_cruise]) / battery_capacity * 100
soc_drop_descend = lpSum([P * delta_t for P in P_descend]) / battery_capacity * 100

single_circuit_drop = soc_drop_taxi + soc_drop_climb + soc_drop_cruise + soc_drop_descend

# Maximum possible value for single_circuit_drop
max_single_circuit_drop = 100 * (power_max_taxi * 0.122 + power_max_climb * 0.033 + power_max_cruise * 0.029 + power_max_descend * 0.05) / battery_capacity

# Link circuit activity to SoC drop using linearization
for circuit in range(1, max_circuits + 1):
    # Constrain soc_drop_actual to match single_circuit_drop when active
    model += soc_drop_actual[circuit] <= max_single_circuit_drop * is_circuit_active[circuit], f"SoC_Drop_UpperBound_{circuit}"
    model += soc_drop_actual[circuit] <= single_circuit_drop, f"SoC_Drop_Match_{circuit}"
    model += soc_drop_actual[circuit] >= single_circuit_drop - (1 - is_circuit_active[circuit]) * max_single_circuit_drop, f"SoC_Drop_LowerBound_{circuit}"
    # Update SoC for each circuit
    model += soc_remaining[circuit] == soc_remaining[circuit - 1] - soc_drop_actual[circuit], f"SoC_Update_Circuit_{circuit}"
    # Ensure SoC is above minimum threshold if circuit is active
    model += soc_remaining[circuit] >= min_soc_for_circuit * is_circuit_active[circuit], f"Min_SoC_Circuit_{circuit}"

# Total number of circuits constraint
model += num_circuits == lpSum([is_circuit_active[c] for c in range(1, max_circuits + 1)]), "Total_Number_of_Circuits"

# Battery capacity constraint
model += lpSum([soc_drop_actual[c] for c in range(1, max_circuits + 1)]) / 100 * battery_capacity <= battery_capacity, "Battery_Capacity"

# Solve the problem
model.solve()

# Print results
print("Status:", model.status)
print("Optimal Number of Circuits:", value(num_circuits))
print("State of Charge After Each Circuit:")
for circuit in range(max_circuits + 1):
    if circuit <= value(num_circuits):
        print(f"Circuit {circuit}: {value(soc_remaining[circuit]):.2f}% SoC")
    else:
        print(f"Circuit {circuit}: Not Performed")

# Total Energy Used
total_energy_used = sum(value(soc_drop_actual[c]) / 100 * battery_capacity for c in range(1, max_circuits + 1))
print(f"\nTotal Energy Used: {total_energy_used:.2f} kWh")


Status: 1
Optimal Number of Circuits: 17.0
State of Charge After Each Circuit:
Circuit 0: 100.00% SoC
Circuit 1: 95.36% SoC
Circuit 2: 90.71% SoC
Circuit 3: 86.07% SoC
Circuit 4: 81.43% SoC
Circuit 5: 76.79% SoC
Circuit 6: 72.14% SoC
Circuit 7: 67.50% SoC
Circuit 8: 62.86% SoC
Circuit 9: 58.21% SoC
Circuit 10: 53.57% SoC
Circuit 11: 53.57% SoC
Circuit 12: 48.93% SoC
Circuit 13: 44.29% SoC
Circuit 14: 39.64% SoC
Circuit 15: 35.00% SoC
Circuit 16: 30.36% SoC
Circuit 17: 25.71% SoC
Circuit 18: Not Performed
Circuit 19: Not Performed
Circuit 20: Not Performed

Total Energy Used: 16.57 kWh


In [70]:
print("\nConstraint Satisfaction Check:")
for name, constraint in model.constraints.items():
    slack = constraint.slack
    print(f"Constraint '{name}': Slack = {slack:.4f}")



Constraint Satisfaction Check:
Constraint 'Initial_SoC': Slack = -0.0000
Constraint 'SoC_Drop_For_Circuit_1': Slack = -0.0000
Constraint 'SoC_Update_Circuit_1': Slack = -0.0000
Constraint 'Min_SoC_Circuit_1': Slack = -69.8651
Constraint 'SoC_Drop_For_Circuit_2': Slack = -0.0000
Constraint 'SoC_Update_Circuit_2': Slack = -0.0000
Constraint 'Min_SoC_Circuit_2': Slack = -59.7302
Constraint 'SoC_Drop_For_Circuit_3': Slack = -0.0000
Constraint 'SoC_Update_Circuit_3': Slack = -0.0000
Constraint 'Min_SoC_Circuit_3': Slack = -49.5952
Constraint 'SoC_Drop_For_Circuit_4': Slack = -0.0000
Constraint 'SoC_Update_Circuit_4': Slack = -0.0000
Constraint 'Min_SoC_Circuit_4': Slack = -39.4603
Constraint 'SoC_Drop_For_Circuit_5': Slack = -0.0000
Constraint 'SoC_Update_Circuit_5': Slack = -0.0000
Constraint 'Min_SoC_Circuit_5': Slack = -29.3254
Constraint 'SoC_Drop_For_Circuit_6': Slack = -0.0000
Constraint 'SoC_Update_Circuit_6': Slack = -0.0000
Constraint 'Min_SoC_Circuit_6': Slack = -19.1905
Constrai